In [253]:
%reload_ext autoreload
%autoreload 2

In [254]:
from pathlib import Path
import sys
from IPython.display import display, HTML
import bpy
import typst
import tempfile

project_root = Path.home() / "projects/blender_typst_importer/"
sys.path.append(str(project_root))


def display_svg(svg_content, width='100px'):
    html_content = (
        "<style>"
        ".svg-container svg { max-width: 100%; height: auto; display: block; }"
        "</style>"
    )
    html_content += f"<div class='svg-container' style='width:{width};'>{svg_content}</div>"
    display(HTML(html_content))

#uv pip install lxml
#uv pip install numpy==1.26.4


temp_dir = Path(tempfile.gettempdir())
#temp_dir = Path.cwd()
typst_file = temp_dir / "step1.typ"
svg_file = temp_dir / "step1.svg"

# Writing example Typst Files

In [255]:
file_content = """
#set page(width: auto, height: auto, margin: 0cm, fill: none)
#set text(size: 50pt)

$ a= b/c $  
"""
typst_file.write_text(file_content)

99

In [256]:
file_content = """
#set page(width: 900pt, height: auto, margin: 0cm, fill: none)

#import "@preview/codelst:2.0.1": sourcecode
#import "@preview/tablex:0.0.8": tablex
#let sourcecode = sourcecode.with(frame: (code) => block(
  radius: 5pt,
  stroke: luma(30),
  inset: 30pt,
  text(size: 30pt, code)
))

#sourcecode[```python
for i in range(0,10):
  print(i)
```]
"""
typst_file.write_text(file_content)

347

In [257]:
file_content = """
#set page(width: auto, height: auto, margin: 0cm, fill: none)
#set text(size: 50pt)

#let korange() = text(fill: orange)[$k$]
#let nblue() = text(fill: blue)[$n$]
$ sum_(#korange() = 1)^#nblue() #korange() = (nblue()(nblue()+1)) / 2 $  
"""
typst_file.write_text(file_content)

238

# Manual SVG conversion + add to scene

In [258]:
from typst_importer.typst_to_svg import simplify_svg, replace_stroke_with_path

typst.compile(typst_file, format="svg", output=str(svg_file))

step1_content = svg_file.read_text()
step2_content = simplify_svg(step1_content)
step3_content = replace_stroke_with_path(step2_content)

display_svg(step1_content , width='500px')
display_svg(step3_content , width='500px')

In [259]:
output_file = temp_dir / "step3.svg"
output_file.write_text(step3_content)

bpy.ops.import_curve.svg(filepath=str(output_file))
col = bpy.context.scene.collection.children['step3.svg']
col.name = "Formula"

for obj in col.objects:
    obj.scale = (100, 100, 100) # this is the old method, better         obj.data.transform(Matrix.Scale(scale_factor, 4))


In [252]:
# Loop through each object in the collection and set its origin
bpy.ops.object.select_all(action='DESELECT')
if col.objects:
    # Set the first object as active
    bpy.context.view_layer.objects.active = col.objects[0]
    # Now we can safely set the mode to OBJECT
    bpy.ops.object.mode_set(mode='OBJECT') 
    for obj in col.objects:
        bpy.context.view_layer.objects.active = obj
        obj.select_set(True)
        bpy.ops.object.origin_set(type='ORIGIN_GEOMETRY', center='MEDIAN')
        obj.select_set(False)

In [240]:
# OR (this can be deleted at some point)
from typst_importer.typst_to_svg import typst_to_blender_curves

collection = typst_to_blender_curves(typst_file)
collection.name = "Debugging_Typst_Collection"

# Using the Package


In [241]:
import tempfile
from pathlib import Path
from typst_importer.typst_to_svg import typst_to_blender_curves
typst_file =  Path(tempfile.gettempdir()) / "hello.typ"

header = """
#set page(width: auto, height: auto, margin: 0cm, fill: none)
#set text(size: 50pt)
"""
body = "$ a= b/c $"
typst_file.write_text(header+body)

collection = typst_to_blender_curves(typst_file)



# Typst Express
(this has a default_header)

In [242]:
from typst_importer.typst_to_svg import typst_express
from typst_importer.curve_utils import shift_scene_content

c = typst_express("$ a = b/d$", scale_factor=100, origin_to_char=False)
shift_scene_content(c)  

In [243]:
content = "$ limits(integral)_a^b f(x) dif x $" 
c = typst_express(content, name="Integral Example")
shift_scene_content(c) 

In [244]:
c = typst_express("""
#let korange() = text(fill: orange)[$k$]
#let nblue() = text(fill: blue)[$n$]
$ sum_(#korange() = 1)^#nblue() #korange() = (nblue()(nblue()+1)) / 2 $  
""")
shift_scene_content(c) 

In [245]:
c = typst_express("""
#set page(width: 900pt, height: auto, margin: 0cm, fill: none)

#import "@preview/codelst:2.0.1": sourcecode
#import "@preview/tablex:0.0.8": tablex
#let sourcecode = sourcecode.with(frame: (code) => block(
  radius: 5pt,
  stroke: luma(30),
  inset: 30pt,
  text(size: 30pt, code)
))

#sourcecode[```python
for i in range(0,10):
  print(i)
```]
""")
shift_scene_content(c, margin=1.2) 

In [246]:
content = """
#set page(width: 900pt, height: auto, margin: 0cm, fill: none)

#import "@preview/codelst:2.0.1": sourcecode
#import "@preview/tablex:0.0.8": tablex

#let sourcecode = sourcecode.with(frame: (code) => block(
  radius: 5pt,
  stroke: luma(30),
  inset: 30pt,
  text(size: 30pt, code)
))

#sourcecode[```python

class HelloWorld:
    def __init__(self):
        self.message = "Hello, world!"

    def greet(self):
        print(self.message)

if __name__ == "__main__":
    hello = HelloWorld()
    hello.greet()
```]
"""
c = typst_express(content)
shift_scene_content(c) 

In [247]:
header = """
#set page(width: auto, height: auto, margin: 0cm, fill: none)
#set text(size: 50pt)
"""
body = "$ a = b/c $"
c = typst_express(header+body)
shift_scene_content(c) 

In [248]:
header = """
#set page(width: auto, height: auto, margin: 0cm, fill: none)
#set text(size: 50pt)
"""
body = "$ limits(integral)_a^b f(x) dif x $"
c = typst_express(header+body)
shift_scene_content(c) 

Info: Deleted 268 object(s)
